This is a port from Python of the Wedding Table Assignment example from [PuLP](https://github.com/coin-or/pulp/blob/master/examples/wedding.py).

In [ ]:
using JuMP, Cbc
using Combinatorics

In [ ]:
function happiness(table):
    """
    Find the happiness of the table
    - by calculating the maximum distance between the letters
    """
    return abs(Int(table[1][1]) - Int(table[end][1]))
end

In [ ]:
max_tables = 5
max_table_size = 4
guests = ["A" "B" "C" "D" "E" "F" "G" "I" "J" "K" "L" "M" "N" "O" "P" "Q" "R"]

In [ ]:
table_combos = [collect(combinations(guests, t)) for t in 1:max_table_size]
;

In [ ]:
# this seems suboptimal, there's probably a better to flatten out these combinations
possible_tables = []
for c in table_combos
    for t in c
        append!(possible_tables, [t])
    end
end

In [ ]:
m = Model(solver=CbcSolver())
num_possible_tables = length(possible_tables)
idx_possible_tables = 1:num_possible_tables

@variable(m, table_assignment[idx_possible_tables], Bin)

# Objective: maximize happiness = minimize happiness value
@objective(m, Min, sum([happiness(possible_tables[t]) * table_assignment[t] for t in idx_possible_tables]))

@constraint(m, sum([table_assignment[t] for t in idx_possible_tables]) <= max_tables)

for guest in guests
    @constraint(m, sum([table_assignment[t] for t in idx_possible_tables if guest in possible_tables[t]]) == 1)
end

;


In [ ]:
print(m)

In [ ]:
status = solve(m)

println("Objective value: ", getobjectivevalue(m))

In [ ]:
table_assignment

In [ ]:
# println("Solution:  ", getvalue(table_assignment))

In [ ]:
for i=1:length(table_assignment)
    if getvalue(table_assignment[i]) == 1
        println(possible_tables[i], " - ", happiness(possible_tables[i]))
    end
end